In [1]:
from hera_mc import geo_sysdef, geo_location, geo_handling
import cartopy.crs as ccrs
from pyuvdata import utils as uvutils
import numpy as np
import pandas as pd

In [2]:
ants = geo_sysdef.read_antennas()
latlon_p = ccrs.Geodetic()
geo_class = geo_handling.Handling()
lat_corr = geo_class.lat_corr[geo_class.hera_zone[1]]
utm_p = ccrs.UTM(geo_class.hera_zone[0])
agl = {}

ant_names = []
ant_nums = []
ant_x = []
ant_y = []
ant_z = []
for ant, ant_info in ants.items():
    ant_names.append(ant)
    ant_nums.append(int(ant[2:]))
    _a = {'station_name': ant,
          'station_type_name': 'hera',
          'easting': ant_info['E'],
          'northing': ant_info['N'],
          'elevation': ant_info['elevation'],
          'datum': 'WGS84',
          'tile': '34J',
          'created_gpstime': 1276288000}
    agl[ant] = geo_location.GeoLocation()
    agl[ant].geo(**_a)
    agl[ant].gps2Time()
    lon, lat = latlon_p.transform_point(agl[ant].easting, agl[ant].northing - lat_corr, utm_p)
    ant_xyz = uvutils.XYZ_from_LatLonAlt(lat * np.pi / 180., lon * np.pi / 180., agl[ant].elevation)
    ant_x.append(ant_xyz[0])
    ant_y.append(ant_xyz[1])
    ant_z.append(ant_xyz[2])



In [3]:
cofa = geo_class.cofa()[0]
telescope_lon, telescope_lat = latlon_p.transform_point(cofa.easting, cofa.northing - lat_corr, utm_p)
telescope_xyz = uvutils.XYZ_from_LatLonAlt(
    telescope_lat * np.pi / 180., telescope_lon * np.pi / 180., cofa.elevation
)

In [4]:
n_ants = len(ant_names)
ant_pos = np.zeros((n_ants, 3), dtype=np.float_)

In [5]:
ant_pos[:,0] = np.array(ant_x) - telescope_xyz[0]
ant_pos[:,1] = np.array(ant_y) - telescope_xyz[1]
ant_pos[:,2] = np.array(ant_z) - telescope_xyz[2]

In [6]:
ant_pos_df = pd.DataFrame(columns = ["names", "numbers", "x", "y", "z"])

In [7]:
ant_pos_df["names"] = ant_names
ant_pos_df["numbers"] = ant_nums
ant_pos_df["x"] = ant_pos[:,0]
ant_pos_df["y"] = ant_pos[:,1]
ant_pos_df["z"] = ant_pos[:,2]

In [8]:
ant_pos_df = ant_pos_df.sort_values("numbers")

In [12]:
ant_pos_df.to_csv("hera_ant_pos.csv", index=False)

In [27]:
columns = ['name', 'number', 'x', 'y', 'z']
formats = ['U10', 'i8', 'f16', 'f16', 'f16']

dt = np.format_parser(formats, columns, [])
ant_array = np.genfromtxt("hera_ant_pos.csv", delimiter=',', autostrip=True, skip_header=1,
                          dtype=dt.dtype)

In [34]:
np.stack((ant_array["x"], ant_array["y"], ant_array["z"])).T.shape

(350, 3)